In [102]:
# Adding all needed imports

import tensorflow as tf
from tensorflow import keras
from keras import regularizers
from keras.models import Model
from keras.layers.convolutional import Conv1D
print(tf.keras.__version__)
print(tf.__version__)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math as mt
import random as rng

from sklearn import preprocessing

2.2.4-tf
2.0.0


In [103]:
dataTrain = np.load("dataFormatted/finished/stTrain.npy", allow_pickle=True)
targetTrain = np.load("dataFormatted/finished/stTrainTarget.npy", allow_pickle=True)


dataVal = np.load("dataFormatted/finished/stVal.npy", allow_pickle=True)
targetVal = np.load("dataFormatted/finished/stValTarget.npy", allow_pickle=True)


print(len(dataTrain[0]))
print(len(targetTrain[0]))
print(len(dataTrain))
print(len(targetTrain))

151
151
80
80


In [104]:
numRows = 30
step = 15



def finalSplit(data, targets, numRows, step):
    div = int(numRows/2)
    splits = []
    splitsTarget = []
    for each in range(len(data)):
        run = data[each]
        target = targets[each]
        for i in range(div, len(target) - div, step):
            split = []
            RUL = 0
            for j in range(-div,div):
                index = i + j
                element = run[index].copy()
                RUL = target[index]
                split.append(element)
            splits.append(split)
            splitsTarget.append(RUL)
            
    return splits, splitsTarget

In [105]:
dataSplitTrain, targetSplitTrain = finalSplit(dataTrain, targetTrain, numRows, step)
dataSplitVal, targetSplitVal = finalSplit(dataVal, targetVal, numRows, step)


print(len(dataSplitTrain))
print(len(dataSplitVal))

712
179


In [106]:
dataSplitTrain = np.array(dataSplitTrain)
targetSplitTrain = np.array(targetSplitTrain)
dataSplitVal = np.array(dataSplitVal)
targetSplitVal = np.array(targetSplitVal)



width = len(dataSplitTrain[0][0])
print(width)

11


In [116]:
visible = layers.Input(shape=(numRows, width))

cnn = layers.Conv1D(width, 3, activation='relu')(visible)
rnn1 = layers.GRU(150, dropout=0.2)(cnn)
#rnn2 = layers.GRU(150, dropout=0.2)(rnn1)


#rnnX = layers.LSTM(300, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(visible)
#rnnY = layers.LSTM(300, dropout=0.2, recurrent_dropout=0.2)(rnnX)

#combo = layers.concatenate([rnn2,rnnY])
dense = layers.Dense(150, activation='relu')(rnn1)
drop = layers.Dropout(0.2)(dense)
out = layers.Dense(1, activation='linear')(drop)
 

# model = Model(inputs=visible, outputs=out)
model = keras.Sequential([
  layers.Conv1D(width, 3, activation='relu'),
  layers.GRU(150, dropout=0.2),
  layers.Dense(150, activation='relu'),
  layers.Dropout(0.2),
  layers.Dense(1, activation='linear')
])

# print(model.summary())


TypeError: The added layer must be an instance of class Layer. Found: Tensor("gru_10/Identity:0", shape=(None, 150), dtype=float32)

In [108]:
model.compile(optimizer='adam',
              loss='mse', 
              metrics=['mae'])


In [109]:
model.fit(dataSplitTrain, targetSplitTrain, epochs=50, verbose=1, validation_data=(dataSplitVal,targetSplitVal))
print(model.summary())

Train on 712 samples, validate on 179 samples
Epoch 1/50
712/712 [==============================] - 2s 3ms/sample - loss: 4405.4521 - mae: 55.1099 - val_loss: 2317.3003 - val_mae: 39.6279
Epoch 2/50
712/712 [==============================] - 0s 572us/sample - loss: 1684.4866 - mae: 35.2902 - val_loss: 1508.1857 - val_mae: 33.7041
Epoch 3/50
712/712 [==============================] - 0s 573us/sample - loss: 1463.9538 - mae: 33.0631 - val_loss: 1414.5222 - val_mae: 32.5188
Epoch 4/50
712/712 [==============================] - 0s 567us/sample - loss: 1359.6850 - mae: 31.8909 - val_loss: 1277.5423 - val_mae: 31.0312
Epoch 5/50
712/712 [==============================] - 0s 621us/sample - loss: 1204.7709 - mae: 29.9234 - val_loss: 1026.5061 - val_mae: 27.7800
Epoch 6/50
712/712 [==============================] - 0s 625us/sample - loss: 902.4195 - mae: 25.9598 - val_loss: 632.4703 - val_mae: 21.3090
Epoch 7/50
712/712 [==============================] - 0s 656us/sample - loss: 535.6976 - mae: 

In [85]:
# model.save('saved_model/final_model')
# model_json = model.to_json()
# with open("./model.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# model.save_weights("model.h5")
# print("Saved model to disk")


In [62]:
# np.save("./valData.npy", simsTest)
# np.save("./valTarget.npy", targetTest)

In [110]:

# import tempfile
# import os

# MODEL_DIR = tempfile.gettempdir()
# version = 1
# export_path = os.path.join(MODEL_DIR, str(version))
# print('export_path = {}\n'.format(export_path))

# tf.keras.models.save_model(
#     model,
#     export_path,
#     overwrite=True,
#     include_optimizer=True,
#     save_format=None,
#     signatures=None,
#     options=None
# )

# print('\nSaved model:')

In [111]:
# import os
# import sys

# export_path_base = sys.argv[-1]
# export_path = os.path.join(
#       tf.compat.as_bytes(export_path_base),
#       tf.compat.as_bytes('1'))
# print('Exporting trained model to', export_path)
# builder = tf.saved_model.builder.SavedModelBuilder(export_path)
# builder.add_meta_graph_and_variables(
#       sess, [tf.saved_model.tag_constants.SERVING],
#       signature_def_map={
#            'predict_images':
#                prediction_signature,
#            signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:
#                classification_signature,
#       },
#       main_op=tf.tables_initializer())
# builder.save()

In [112]:
# model.save(model, "/tmp/finalModel/1/", save_format="tf")

In [113]:
# keras.experimental.export_saved_model(model,'saved_model/3/')

In [115]:
tf.saved_model.save(model, '/saved_model/1/')

INFO:tensorflow:Assets written to: /saved_model/1/assets
